# Recording Data Provenance with ``aiida-gromacs``

This tutorial follows Justin Lemkul’s [lysozyme tutorial](http://www.mdtutorials.com/gmx/lysozyme/). We will not explain each individual step as this can be found on Justin’s webpage, but we will link to each page and show the AiiDA equivalant command.

Please also note the slight differences in commands between the tutorial and that by Justin Lemkul is simply down to the way we are recording provenance requires non-interactive input into the gromacs tools.

Also at each of the below steps you should run ``verdi`` (the command line interface utility for AiiDA) to view the status of the submitted process before moving onto the next step, you do this by:

In [1]:
! verdi process list -a

PK    Created    Process label    Process State    Process status
----  ---------  ---------------  ---------------  ----------------

Total results: 0

Report: last time an entry changed state: 5D ago (at 16:57:13 on 2023-10-05)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


In this notebook, AiiDA, the aiida-gromacs plugin and dependent tools are pre-installed. Here's a brief description of the tools used; AiiDA uses a [PostgreSQL](https://www.postgresql.org) database to store all data produced and the links between input and output files for each command run, which are termed processes in AiiDA. Communication between submitted processes are handled with [RabbitMQ](https://www.rabbitmq.com/) and submitted processes are handled with a deamon process that runs in the background. 

A successfully finished process will exit with code [0]. 

1. We will start from the [pbd2gmx](http://www.mdtutorials.com/gmx/lysozyme/01_pdb2gmx.html)  step of Justin’s tutorial, with the aiida-gromacs equivalent python wrapper ``gmx_pdb2gmx``:

In [4]:
! gmx_pdb2gmx -f gromacs_files/1AKI_clean.pdb -ff oplsaa -water spce -o 1AKI_forcefield.gro -p 1AKI_topology.top -i 1AKI_restraints.itp

In [3]:
! verdi process list -a

  PK  Created    Process label       Process State    Process status
----  ---------  ------------------  ---------------  ----------------
 541  3s ago     Pdb2gmxCalculation  ⏹ Finished [0]

Total results: 1

Report: last time an entry changed state: 2s ago (at 20:44:43 on 2023-10-10)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


2. Next we will create the [box and then solvate](http://www.mdtutorials.com/gmx/lysozyme/03_solvate.html):

In [16]:
! gmx_editconf -f 1AKI_forcefield.gro -center 0 -d 1.0 -bt cubic -o 1AKI_newbox.gro
! gmx_solvate -cp 1AKI_newbox.gro -cs spc216.gro -p 1AKI_topology.top -o 1AKI_solvated.gro

In [ ]:
! verdi process list -a

3. & 4. Then add [ions](http://www.mdtutorials.com/gmx/lysozyme/04_ions.html) to neutralise the system after preprocessing the topology:

In [19]:
! gmx_grompp -f gromacs_files/ions.mdp -c 1AKI_solvated.gro -p 1AKI_topology.top -o 1AKI_ions.tpr
! gmx_genion -s 1AKI_ions.tpr -p 1AKI_topology.top -pname NA -nname CL -neutral true -o 1AKI_solvated_ions.gro

5. & 6. The [minimise](http://www.mdtutorials.com/gmx/lysozyme/05_EM.html) the system after preprocessing the topology:

In [ ]:
! gmx_grompp -f gromacs_files/min.mdp -c 1AKI_solvated_ions.gro -p 1AKI_topology.top -o 1AKI_minimised.tpr
! gmx_mdrun -s 1AKI_minimised.tpr -c 1AKI_minimised.gro -e 1AKI_minimised.edr -g 1AKI_minimised.log -o 1AKI_minimised.trr

In [ ]:
! verdi node graph generate 534